# 03 클러스터 환경

## 3.1 클러스터 환경

클러스터 모드 : 여러 대의 서버에서 동작하는 방식.

클러스터 환경에서는 여러 서버를 마치 하나의 서버인 것처럼 다루어야 함. 분산 작업 관리 기능이 추가됨.

### 3.1.1 클러스터 모드와 컴포넌트

클러스터 매니저 : 전체 서버의 자원과 동작을 세밀하고 효율적으로 제어할 수 있는 모듈.

잡(Job) : 전체 작업 중에 포함된 세부 작업을 가리키는 용도.

드라이버 : 스파크 컨텍스트를 생성하고 관리하는 프로그램.

각 워커 노드에는 익스큐터라는 스파크 프로세스가 구동되면서 작업을 수행.

<img src="cluster-overview.png">

호스트,서버 : 주로 물리적인 장비로서의 의미.

노드 : 프로그래밍 모델 관점에서 작업이 수행되는 끝점.

마스터, OO매니저 : 클러스터 운영 및 제어에 사용되는 서버

슬레이브, 워커, 작업노드 등 : 실제 작업에 사용되는 서버.

스파크 컨텍스트 : 스파크 클러스와의 연결.

모든 스파크 애플리케이션은 스파크컨텍스트를 생성으로부터 시작. 

1개 이상 스파크컨텍스트를 생성하거나 다른 애플리케이션이 생성한 스파크컨텍스트를 공유할 수 없음.

잡은 액션 연산수 만큼 생성. 잡은 스테이지라는 단계로 나누어 실행. (셔플을 기준으로 나눔)

익스큐터 역할 : 할당받은 태스크 처리. 재사용을 위해 데이터를 메모리에 저장. 

### 3.1.2 클러스터 모드를 위한 시스템 구성

1. 개발서버에서 애플리케이션 코드를 작성. 테스트 수행
2. 배포에 필요한 압축파일(jar/war, zip, egg)을 만듬
3. 실행서버에 배포하고 작업 실행. spark-submit 스크립트 혹은 스파크 셸 스크립트를 통해 수행.

배치서버/클라이언트 서버 : 클러스터에 작업을 요청하는 서버.

[PC] = 1. 배포 => 배치 서버/클라이언트 서버 = 2. 작업제출 실행 / 3. 결과 취합/처리 => 스파크 클러스터

**기본준비 사항**

* 네트워크 설정 
 - 모든 서버들이 ssh, rsync로 파일 교환이 가능하도록 설정.
 - https://github.com/SDRLurker/learn_spark/blob/master/spark_install/ssh.sh
* JAVA_HOME 설정
* HADOOP_HOME 설정
 - 하둡 바이너리 설치
 - https://github.com/SDRLurker/learn_spark/blob/master/spark_install/hadoop.sh
* 하둡 파일시스템(HDFS)
 - 데이터의 입력 및 출력을 위한 파일시스템으로 HDFS 사용. (아래는 hdfs 멀티노드용 설정)
 - https://github.com/SDRLurker/learn_spark/blob/master/spark_install/hd_multi.sh
 
**하둡 파일시스템 실습 예제**

* 하둡 파일시스템 새로운 디렉터리 생성

```shell
$ hdfs dfs -mkdir hdfs://master:9000/sample
```

* $SPARK_HOME/README.md를 방금 생성한 디렉터리로 업로드

```shell
$ cd ${SPARK_HOME}
$ hdfs dfs -put ./README.md hdfs://master:9000/sample
```

* ls 명령으로 파일이 잘 업로드 되었는지 확인

```shell
$ hdfs dfs -ls hdfs://master:9000/sample
Found 1 items
-rw-r--r--   3 spark supergroup       3809 2017-10-29 02:40 hdfs://master:9000/sample/README.md
```

**로컬 개발서버**

* 스파크 셸을 이용 (일반적인 방법)
* 메인 함수를 가진 애플리케이션을 직접 실행
* 메인 함수를 가진 애플리케이션을 spark-submit 스크립트를 이용해 실행

**애플리케이션 실행 서버**

spark-submit, spark-shell 등의 스크립트를 이용해 스파크 애플리케이션을 맨처음 실행하는 서버.

#### 클러스터 서버

운영을 위한 마스터 서버

실제 데이터를 처리하고 필요에 따라 저장하는 워커 노드의 역할을 수행하는 서버.

### 3.1.3 드라이버 프로그램과 디플로이 모드

드라이버 프로그램 : 스파크 컨텍스트를 생성하는 코드가 포함된 프로그램

클러스터에서 실행할 때는 클러스터 매니저에게 애플리케이션 실행을 요청하는 방식을 사용.

**디플로이 모드**

클라이언트 디플로이 모드 : 애플리케이션을 실행한 프로세스 내부에서 드라이버 프로그램을 구동. 

* 드라이버 프로그램(SparkContext) ⊆ 애플리케이션

클러스터 디플로이 모드 : 클러스터 매니저에게 작업 실행만 요청. 실제 드라이버 프로그램의 실행은 클러스터 내부에서 실행.

* 클러스터 매니저 <=> 드라이버 프로그램(SparkContext) ⊆ 노드 

## 3.2 클러스터 매니저

클러스터 매니저 : 클러스터 환경에서 다수의 애플리케이션이 함께 구동될 수 있게 애플리케이션 간의 CPU나 메모리, 디스크와 같은 컴퓨팅 자원을 관리.

* 하둡의 얀(Yarn)
* 아파치의 메소스(Mesos)
* 스탠드얼론(Standalone) : 스파크에서 자체적으로 제공.

### 3.2.1 스탠드얼론 클러스터 매니저

#### 3.2.1.1 개요

**마스터 인스턴스**

* 클러스터 매니저 컴포넌트.
* 클라이언트의 요청을 받아 필요한 서버 자원을 할당.

**슬레이브 인스턴스**

* 워커 노드에 해당.
* 익스큐터(Executor)와 태스크(Task)를 실행해 데이터에 대한 실제 처리와 저장을 수행하는 역할.

#### 3.2.1.2 설치

**기본준비 사항**

* 서버 준비 및 기본 설정
 - master : CPU 코어 4개, 메모리 8GB
 - slave 4대 : CPU 코어 1개, 메모리 2GB
* ssh 접속 설정.
 - 모든 서버들이 ssh, rsync로 파일 교환이 가능하도록 설정. (하둡과 동일해서 생략가능)
 - https://github.com/SDRLurker/learn_spark/blob/master/spark_install/ssh.sh
* SCALA 설치 및 SCALA_HOME 설정.
 - https://github.com/SDRLurker/learn_spark/blob/master/spark_install/scala.sh
* SPARK 설치 및 SPARK_HOME 설정
 - 스파크 바이너리 설치. 모든 서버에 설치함.
 - https://github.com/SDRLurker/learn_spark/blob/master/spark_install/spark.sh
* slave 서버 등록.
 - start-all.sh 실행스크립트를 통해 마스터와 슬레이브를 한 번에 실행할 때 사용.
 - 어떤 서버에서 슬레이브 프로세스를 시작해야하는지 알려주어야 함.
 - https://github.com/SDRLurker/learn_spark/blob/master/spark_install/setup.sh

#### 3.2.1.3 클러스터 매니저 실행

$SPARK_HOME/sbin 디렉터리로 이동하면 다음 스크립트가 있음.

```shell
$ cd ${SPARK_HOME}/sbin
```

**매니저 서버 개별 실행과 종료**

* start-master.sh / stop-master.sh : 마스터 인스턴스 실행 / 종료
* start-slave.sh <마스터_url> : 슬레이브 인스턴스 실행. 예시) 
 - ./start-slave.sh master:7077
* stop-slave.sh : 슬레이브 인스턴스 종료.

앞에서 정의한 slaves 파일의 내용은 아무런 영향이 없음.

**한 번의 실행으로 다수의 프로세스를 구동시키고 종료**

* start-slaves.sh / stop-slaves.sh : conf/slaves 파일에 등록된 서버에 슬레이브 인스턴스를 실행 / 종료
* start-all.sh / stop-all.sh : 마스터와 슬레이브 인스턴스(conf/slaves 참조)를 모두 실행 / 종료

** start-master.sh, stop-master.sh 옵션 **

* -h HOST, --host HOST : 인스턴스에 접속할 호스트명을 지정
* -p PORT, --port PORT : 마스터, 슬레이브에 접속할 포트명 지정. 마스터의 경우 디폴트 7077
* --webui-port PORT : 웹브라우저를 통해 마스터 또는 슬레이브에 접속할 때 사용할 포트 지정. 마스터 디폴트 8080, 슬레이브는 8081.
* -c CORES, --cores CORES : 스파크 애플리케이션이 사용 가능한 최대 코어수를 지정. 슬레이브만 지정가능. 디폴트는 가용한 전체코어가 할당.
* -m MEM, --memory MEM : 스파크 애플리케이션이 사용 가능한 메모리를 지정. 슬레이브만 지정가능. 디폴트는 최대메모리-1GB.

**스파크 마스터/슬레이브 한꺼번에 실행**

```shell
[spark@master ~]$ cd $SPARK_HOME/sbin
[spark@master sbin]$ ./start-all.sh
```

정상적으로 실행되었는지 다음처럼 http://master:8080 으로 접속하여 확인가능.

In [1]:
import requests
r = requests.get("http://master:8080")

from IPython.display import HTML
HTML(r.text)

* URL: spark://master:7077
 - 마스터 서버의 접속주소
* Alive Workers: 4
 - 클러스터의 워커노드 수

#### 3.2.1.4 애플리케이션 실행서버 준비

여기서는 master 서버를 애플리케이션 실행서버로 사용.

책은 svr-client 서버로 애플리케이션 실행서버로 사용.

#### 3.2.1.5 애플리케이션 실행

##### 3.2.1.5.1 스파크 셸

--master 옵션으로 클러스터 마스터 URL 지정.

```shell
[spark@master ~]$ $SPARK_HOME/bin/spark-shell --master spark://master:7077
```

정상적으로 수행되면 'Running Applications'에 새로운 애플리케이션이 등록이 됨.

In [2]:
from bs4 import BeautifulSoup

r = requests.get("http://master:8080")
soup = BeautifulSoup(r.text, "html.parser")
HTML(str(soup.find("h4", {"id":"running-app"}).parent))

app-20171029114018-0002 (kill),Spark shell,16,1024.0 MB,2017/10/29 11:40:18,spark,RUNNING,2.7 min


옵션을 지정하지 않을 경우 메모리는 기본 1Gb, 워커 노드는 최대 가능한 코어수로 할당.

hdfs의 README.md 단어 수를 세는 예제.

```scala
scala> val inputPath = "hdfs://master:9000/sample/README.md"
scala> val outputPath = "hdfs://master:9000/sample/output"
scala> sc.textFile(inputPath) flatMap { line => line.split(" ") map (word => (word, 1L)) } reduceByKey (_ + _) saveAsTextFile (outputPath)
```

결과확인

In [3]:
%%bash
hdfs dfs -cat hdfs://master:9000/sample/output/p* | head

(package,1)
(this,1)
(Version"](http://spark.apache.org/docs/latest/building-spark.html#specifying-the-hadoop-version),1)
(Because,1)
(Python,2)
(page](http://spark.apache.org/documentation.html).,1)
(cluster.,1)
([run,1)
(its,1)
(YARN,,1)


cat: Unable to write to output stream.


##### 3.2.1.5.2 pyspark

--master 옵션으로 클러스터 마스터 URL 지정.

```shell
[spark@master ~]$ $SPARK_HOME/bin/pyspark --master spark://master:7077
```

정상적으로 수행되면 'Running Applications'에 새로운 애플리케이션이 등록이 됨.

In [4]:
r = requests.get("http://master:8080")
soup = BeautifulSoup(r.text, "html.parser")
HTML(str(soup.find("h4", {"id":"running-app"}).parent))

app-20171029114649-0003 (kill),PySparkShell,16,1024.0 MB,2017/10/29 11:46:49,spark,RUNNING,31 s


정상적으로 수행되면 'Running Applications'에 "PySparkShell"이 구동중인 것을 확인가능.

hdfs의 README.md 단어 수를 세는 예제.

```python
>>> inputPath = "hdfs://master:9000/sample/README.md"
>>> outputPath = "hdfs://master:9000/sample/output"
>>> result = rdd.flatMap(lambda line: line.split(" ")).map(lambda word:(word,1L)).reduceByKey(lambda v1, v2: v1+v2)
>>> result.saveAsTextFile(outputPath)
```

In [5]:
%%bash
hdfs dfs -cat hdfs://master:9000/sample/output/p* | head

(u'', 71L)
(u'storage', 1L)
(u'"local"', 1L)
(u'including', 4L)
(u'computation', 1L)
(u'file', 1L)
(u'Maven', 1L)
(u'using:', 1L)
(u'guidance', 2L)
(u'Scala,', 1L)


cat: Unable to write to output stream.


##### 3.2.1.5.3 spark-submit을 이용한 애플리케이션 실행

배포 파일을 만들고 이를 스파크 클러스터에 디플로이해서 실행하는 것이 더 편리함.

일관된 방법으로 애플리케이션을 실행가능하며 실행 매개변수를 이용해 각각 다르게 설정 가능.

애플리케이션 코드를 작성한 뒤 jar 파일을 생성. (파이썬의 경우 .py나 zip, egg)

스칼라로 작성한 단어 수 세기 예제

```scala
package com.wikibooks.spark.ch3.scala

import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.rdd.RDD.rddToPairRDDFunctions

object WordCount {

  def run(inputPath: String, outputPath: String) {
    val conf = new SparkConf()
    val sc = new SparkContext(conf)

    sc.textFile(inputPath)
      .flatMap(_.split(" "))
      .map((_, 1L))
      .reduceByKey(_ + _)
      .saveAsTextFile(outputPath)

    sc.stop()
  }

  def main(args: Array[String]) {
    val numberOfArgs = 2
    if (args.length == numberOfArgs) {
      run(args(0), args(1))
    } else {
      println("Usage: $SPARK_HOME/bin/spark-submit --class <class_name> --master <master> --<option> <option_value> <jar_file_path> <input_path> <output_path>")
    }
  }
}
```

메이븐으로 jar 배포용 파일을 생성
```shell
[spark@master ~]$ git clone https://github.com/wikibook/spark.git
[spark@master ~]$ cd spark
[spark@master spark]$ mvn -Dmaven.test.skip=true clean package
```

~/sparkApps 디렉터리 생성 및 jar 파일을 복사
```shell
[spark@master spark]$ mkdir -p ~/sparkApps
[spark@master spark]$ cp -p ../deploy/beginning-spark-examples.jar ~/sparkApps
```

하둡파일 시스템에 이전 출력 파일 삭제
```shell
[spark@master spark]$ hdfs dfs -rm -r hdfs://master:9000/sample/output
```

스파크 애플리케이션을 실행
```shell
[spark@master spark]$ cd $SPARK_HOME
[spark@master spark]$ ./bin/spark-submit --class com.wikibooks.spark.ch3.scala.WordCount --master spark://master:7077 ~/sparkApps/beginning-spark-examples.jar hdfs://master:9000/sample/README.md hdfs://master:9000/sample/output
```

결과 확인

In [6]:
%%bash
hdfs dfs -cat hdfs://master:9000/sample/output/p* | head

(package,1)
(this,1)
(Version"](http://spark.apache.org/docs/latest/building-spark.html#specifying-the-hadoop-version),1)
(Because,1)
(Python,2)
(page](http://spark.apache.org/documentation.html).,1)
(cluster.,1)
([run,1)
(its,1)
(YARN,,1)


cat: Unable to write to output stream.


파이썬으로 작성한 단어 수 세기 예제

```python
from pyspark import SparkContext, SparkConf
import sys

class WordCount:    
    def run(self, inputPath, outputPath):
        conf = SparkConf()        
        sc = SparkContext(conf=conf)
        sc.textFile(inputPath).flatMap(lambda s : s.split(" ")).map(lambda s: (s, 1)).reduceByKey(lambda v1, v2:v1 + v2).saveAsTextFile(outputPath)
        sc.stop()
        
if __name__ == "__main__":
    app = WordCount()
    
    if len(sys.argv) != 3:
        print("Usage: $SPARK_HOME/bin/spark-submit --master <master> --<option> <option_value> <python_file_path> <input_path> <output_path>")
    else:
        app.run(sys.argv[1], sys.argv[2])
```

실행방법
```shell
[spark@master ~]$ spark-submit --master spark://master:7077 $HOME/spark/Python/ch3/wordcount.py hdfs://master:9000/README.md hdfs://master:9000/output
```

결과확인

In [9]:
%%bash
hdfs dfs -cat hdfs://master:9000/sample/output/p* | head

(u'', 71)
(u'when', 1)
(u'R,', 1)
(u'including', 4)
(u'computation', 1)
(u'contributing', 1)
(u'using:', 1)
(u'guidance', 2)
(u'Scala,', 1)
(u'environment', 1)


cat: Unable to write to output stream.


#### 3.2.1.6 디플로이 모드

--deploy-mode 옵션을 사용해 "cluster"와 "client" 모드 중 하나를 지정 가능.

기본값은 "client" 모드.

배포 jar 파일을 slave에 복사.(책에 없음)
```shell
[spark@master ~]$ scp -rp sparkApps/ slave01:~/.
[spark@master ~]$ scp -rp sparkApps/ slave02:~/.
[spark@master ~]$ scp -rp sparkApps/ slave03:~/.
[spark@master ~]$ scp -rp sparkApps/ slave04:~/.
```

스파크 애플리케이션을 "cluster" 모드에서 실행
```shell

[spark@master ~]$ cd $SPARK_HOME
[spark@master spark]$ ./bin/spark-submit --class com.wikibooks.spark.ch3.scala.WordCount --master spark://master:7077 --deploy-mode cluster ~/sparkApps/beginning-spark-examples.jar hdfs://master:9000/sample/README.md hdfs://master:9000/sample/output
```

#### 3.2.1.7 주요 설정

conf의 spark-env.sh 파일을 만들고 "export 변수=값" 형태로 환경변수를 저장.

자세한 내용은 https://goo.gl/hrxard 참조

**마스터 및 슬레이브 실행 스크립트의 명령행 매개변수**

* --properties-file : 스파크 설정 파일의 위치를 지정. 기본으로는 spark-defaults.conf 파일 사용.

**주요 환경변수(모든 스크립트에 적용)**

* SPARK_WORKER_MEMORY : 스파크 애플리케이션이 사용 가능한 메모리를 지정.